# Policy Gradient

## 基于时序差分的行动-批判算法

In [1]:
import gym
import torch
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import rl_utils

In [2]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)

In [3]:
class ValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim):
        super(ValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)

In [4]:
class ActorCritic:
    def __init__(self, state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                 gamma, device):
        # 策略网络
        self.actor = PolicyNet(state_dim, hidden_dim, action_dim).to(device)
        self.critic = ValueNet(state_dim, hidden_dim).to(device)  # 价值网络
        # 策略网络优化器
        self.actor_optimizer = torch.optim.Adam(self.actor.parameters(),
                                                lr=actor_lr)
        self.critic_optimizer = torch.optim.Adam(self.critic.parameters(),
                                                 lr=critic_lr)  # 价值网络优化器
        self.gamma = gamma
        self.device = device

    def take_action(self, state):
        state = torch.tensor([state], dtype=torch.float).to(self.device)
        probs = self.actor(state)
        action_dist = torch.distributions.Categorical(probs)
        action = action_dist.sample()
        return action.item()

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'],
                              dtype=torch.float).to(self.device)
        actions = torch.tensor(transition_dict['actions']).view(-1, 1).to(
            self.device)
        rewards = torch.tensor(transition_dict['rewards'],
                               dtype=torch.float).view(-1, 1).to(self.device)
        next_states = torch.tensor(transition_dict['next_states'],
                                   dtype=torch.float).to(self.device)
        dones = torch.tensor(transition_dict['dones'],
                             dtype=torch.float).view(-1, 1).to(self.device)

        # 时序差分目标
        td_target = rewards + self.gamma * self.critic(next_states) * (1 -
                                                                       dones)
        td_delta = td_target - self.critic(states)  # 时序差分误差
        log_probs = torch.log(self.actor(states).gather(1, actions))
        actor_loss = torch.mean(-log_probs * td_delta.detach())
        # 均方误差损失函数
        critic_loss = torch.mean(
            F.mse_loss(self.critic(states), td_target.detach()))
        self.actor_optimizer.zero_grad()
        self.critic_optimizer.zero_grad()
        actor_loss.backward()  # 计算策略网络的梯度
        critic_loss.backward()  # 计算价值网络的梯度
        self.actor_optimizer.step()  # 更新策略网络的参数
        self.critic_optimizer.step()  # 更新价值网络的参数

In [5]:
actor_lr = 1e-3
critic_lr = 1e-2
num_episodes = 1000
hidden_dim = 128
gamma = 0.98
device = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

env_name = 'CartPole-v1'
env = gym.make(env_name)
env.reset(seed=0)
torch.manual_seed(0)
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
agent = ActorCritic(state_dim, hidden_dim, action_dim, actor_lr, critic_lr,
                    gamma, device)

return_list = rl_utils.train_on_policy_agent(env, agent, num_episodes)

Iteration 0:   0%|          | 0/100 [00:00<?, ?it/s]/var/folders/98/yrg96r1957xgshg2r4rfvh180000gn/T/ipykernel_50160/2458400454.py:17: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1702400277931/work/torch/csrc/utils/tensor_new.cpp:264.)
  state = torch.tensor([state], dtype=torch.float).to(self.device)
/Users/jiangyongwei/anaconda3/envs/monai/lib/python3.11/site-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
Iteration 0:   1%|          | 1/100 [00:00<00:16,  5.94it/s]

(array([0.03132702, 0.04127556, 0.01066358, 0.02294966], dtype=float32), {})
[array([ 0.0043625 ,  0.04350724,  0.03158535, -0.04972615], dtype=float32)]
[array([ 0.00523264,  0.23816238,  0.03059083, -0.33227867], dtype=float32)]
[array([ 0.00999589,  0.43283588,  0.02394526, -0.61516   ], dtype=float32)]
[array([ 0.01865261,  0.6276152 ,  0.01164206, -0.90020597], dtype=float32)]
[array([ 0.03120491,  0.43233743, -0.00636206, -0.60388654], dtype=float32)]
[array([ 0.03985166,  0.6275478 , -0.01843979, -0.89856654], dtype=float32)]
[array([ 0.05240262,  0.43268055, -0.03641112, -0.61173636], dtype=float32)]
[array([ 0.06105623,  0.23808591, -0.04864585, -0.3307404 ], dtype=float32)]
[array([ 0.06581794,  0.43386537, -0.05526066, -0.63835835], dtype=float32)]
[array([ 0.07449526,  0.6297125 , -0.06802782, -0.9479191 ], dtype=float32)]
[array([ 0.08708951,  0.43556926, -0.08698621, -0.6773625 ], dtype=float32)]
[array([ 0.09580089,  0.24175666, -0.10053346, -0.4132845 ], dtype=float32)]

Iteration 0:  15%|█▌        | 15/100 [00:00<00:01, 43.75it/s, episode=10, return=13.700]

[array([-0.10630573, -0.96221286,  0.13547166,  1.6493179 ], dtype=float32)]
[array([-0.12554999, -0.76890963,  0.16845801,  1.4017267 ], dtype=float32)]
[array([-0.14092818, -0.96567595,  0.19649254,  1.7419909 ], dtype=float32)]
(array([-0.00141646,  0.03894878,  0.04340435, -0.01422048], dtype=float32), {})
[array([ 0.00715298, -0.01781306,  0.00943   , -0.01620888], dtype=float32)]
[array([ 0.00679672, -0.21306898,  0.00910583,  0.27943435], dtype=float32)]
[array([ 0.00253534, -0.01807809,  0.01469451, -0.01036277], dtype=float32)]
[array([ 0.00217378, -0.21340767,  0.01448726,  0.28691998], dtype=float32)]
[array([-0.00209437, -0.4087332 ,  0.02022566,  0.58413666], dtype=float32)]
[array([-0.01026904, -0.60413253,  0.03190839,  0.88312167], dtype=float32)]
[array([-0.02235169, -0.79967296,  0.04957082,  1.1856625 ], dtype=float32)]
[array([-0.03834515, -0.99540156,  0.07328407,  1.4934626 ], dtype=float32)]
[array([-0.05825318, -0.80124366,  0.10315333,  1.2245337 ], dtype=float

Iteration 0:  20%|██        | 20/100 [00:00<00:01, 43.68it/s, episode=20, return=19.800]

...

Iteration 3:   5%|▌         | 5/100 [00:00<00:12,  7.66it/s]

[array([ 0.60618055,  1.4405813 ,  0.01197403, -0.3741386 ], dtype=float32)]
[array([ 0.6349922 ,  1.6355312 ,  0.00449125, -0.6630221 ], dtype=float32)]
[array([ 0.6677028 ,  1.440347  , -0.00876919, -0.3689284 ], dtype=float32)]
[array([ 0.6965098 ,  1.2453507 , -0.01614776, -0.07902338], dtype=float32)]
[array([ 0.7214168 ,  1.4407004 , -0.01772822, -0.3767569 ], dtype=float32)]
[array([ 0.7502308 ,  1.6360697 , -0.02526336, -0.67497647], dtype=float32)]
[array([ 0.7829522 ,  1.4413077 , -0.03876289, -0.39035347], dtype=float32)]
[array([ 0.8117783 ,  1.2467567 , -0.04656996, -0.11013965], dtype=float32)]
[array([ 0.83671343,  1.442514  , -0.04877276, -0.41714382], dtype=float32)]
[array([ 0.86556375,  1.638292  , -0.05711563, -0.7247951 ], dtype=float32)]
[array([ 0.89832956,  1.8341553 , -0.07161153, -1.0348939 ], dtype=float32)]
[array([ 0.9350127 ,  1.6400548 , -0.09230941, -0.76552516], dtype=float32)]
[array([ 0.9678138 ,  1.446317  , -0.10761991, -0.5032561 ], dtype=float32)]

Iteration 3:   5%|▌         | 5/100 [00:00<00:16,  5.91it/s]

[array([ 0.46594086,  0.39793703,  0.00349285, -0.04052249], dtype=float32)]
[array([ 0.4738996 ,  0.5930087 ,  0.0026824 , -0.33210135], dtype=float32)]
[array([ 0.4857598 ,  0.39784867, -0.00395963, -0.03857373], dtype=float32)]
[array([ 0.49371678,  0.20278373, -0.0047311 ,  0.25285727], dtype=float32)]
[array([ 4.9777243e-01,  3.9797291e-01,  3.2604265e-04, -4.1314170e-02],
      dtype=float32)]
[array([ 5.057319e-01,  5.930902e-01, -5.002407e-04, -3.338942e-01],
      dtype=float32)]
[array([ 0.5175937 ,  0.39797536, -0.00717812, -0.04136907], dtype=float32)]
[array([ 0.5255532 ,  0.20295708, -0.00800551,  0.24904047], dtype=float32)]
[array([ 0.52961236,  0.39819244, -0.0030247 , -0.04615678], dtype=float32)]
[array([ 0.5375762 ,  0.5933576 , -0.00394783, -0.3397925 ], dtype=float32)]
[array([ 0.54944336,  0.78853554, -0.01074368, -0.6337177 ], dtype=float32)]
[array([ 0.56521404,  0.5935651 , -0.02341804, -0.3444375 ], dtype=float32)]
[array([ 0.5770854 ,  0.39878395, -0.0303067

KeyboardInterrupt: 